In [22]:
ls /home/gsm/Documents/extras/kaggle/petfinder/

breed_labels.csv  color_labels.csv  state_labels.csv  test/  train/


In [179]:
#cat /home/gsm/Documents/extras/kaggle/petfinder/state_labels.csv

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [136]:
df = pd.read_csv('/home/gsm/Documents/extras/kaggle/petfinder/train/train.csv')
#drop_names = ['RescuerID','FurLength','Name','Color3']
#ddf = df.drop(columns=drop_names)

In [182]:
df.groupby('AdoptionSpeed').RescuerID.value_counts().unstack().max(axis=1)

AdoptionSpeed
0     12.0
1     52.0
2    173.0
3    223.0
4     90.0
dtype: float64

In [137]:
df.head(3)

,Type,Name,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,...,Health,Quantity,Fee,State,RescuerID,VideoAmt,Description,PetID,PhotoAmt,AdoptionSpeed
0,2,Nibble,3,299,0,1,1,7,0,1,...,1,1,100,41326,8480853f516546f6cf33aa88cd76c379,0,Nibble is a 3+ month old ball of cuteness. He ...,86e1089a3,1.0,2
1,2,No Name Yet,1,265,0,1,1,2,0,2,...,1,1,0,41401,3082c7125d8fb66f7dd4bff4192c8b14,0,I just found it alone yesterday near my apartm...,6296e909a,2.0,0
2,1,Brisco,1,307,0,1,2,7,0,2,...,1,1,0,41326,fa90fa5b1ee11c86938398b60abc32cb,0,Their pregnant mother was dumped by her irresp...,3422e4906,7.0,3


In [140]:
#TODO clean up Descriptions nan's with  some imputer in text pipeline
df.Description.fillna('missing',inplace=True)

In [188]:
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline

from sklearn.preprocessing import Binarizer,MinMaxScaler,StandardScaler
from sklearn.preprocessing import OneHotEncoder,QuantileTransformer
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_transformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier as DTC
from sklearn.ensemble import RandomForestClassifier as RFC

from sklearn.metrics import classification_report
from sklearn.metrics import cohen_kappa_score,make_scorer

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

from lightgbm import LGBMClassifier
from xgboost import XGBClassifier


In [189]:
lgbm = LGBMClassifier(objective='multiclass',**params)
params = {#'application': 'regression',
          'boosting': 'gbdt',
          'metric': 'cohen_kappa_score',
          'num_leaves': 70,
          'max_depth': 9,
          'learning_rate': 0.01,
          'bagging_fraction': 0.85,
          'feature_fraction': 0.8,
          'min_split_gain': 0.02,
          'min_child_samples': 150,
          'min_child_weight': 0.02,
          'lambda_l2': 0.0475,
          'verbosity': -1}
xgb = XGBClassifier(metric='cohen_kappa_score')
svc = SVC(C=1.)

In [197]:
numeric_features = ['Age','Health','Dewormed','Vaccinated','Fee','PhotoAmt']
numeric_transformer = Pipeline(steps=[
    #('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_features = ['State','Sterilized','Type','Breed1','Gender']
categorical_transformer = Pipeline(steps=[
    #('imputer', SimpleImputer(strategy='constant', fill_value=0)),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

text_features = 'Description'
text_transformer = Pipeline(steps=[
    #('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('bow', TfidfVectorizer(min_df=2,  max_features=None,
        strip_accents='unicode', analyzer='word', token_pattern=r'(?u)\b\w+\b',
        ngram_range=(1, 2), use_idf=1, smooth_idf=1, sublinear_tf=1,)),
    ('svd',TruncatedSVD(n_components=100))

])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features),
        ('txt', text_transformer,text_features)],
    remainder='drop')

clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', xgb)])

X = ddf.drop('AdoptionSpeed', axis=1)
y = ddf['AdoptionSpeed']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,shuffle=True)

clf.fit(X_train, y_train)

/home/gsm/miniconda3/envs/dsml/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/gsm/miniconda3/envs/dsml/lib/python3.6/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)


Pipeline(memory=None,
     steps=[('preprocessor', ColumnTransformer(n_jobs=None, remainder='drop', sparse_threshold=0.3,
         transformer_weights=None,
         transformers=[('num', Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True))]), ['Age', 'Health', 'Dewormed', 'Va... reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=1))])

In [198]:
yh = clf.predict(X_test)
#print("model score: %.3f" % clf.score(X_test, y_test))

/home/gsm/miniconda3/envs/dsml/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


In [199]:
cohen_kappa_score(y_test,yh,weights='quadratic')

0.3057212344055046

In [200]:
print(classification_report(yh,y_test))

              precision    recall  f1-score   support

           0       0.02      1.00      0.04         2
           1       0.22      0.36      0.27       392
           2       0.44      0.33      0.38      1051
           3       0.22      0.43      0.29       325
           4       0.68      0.47      0.56      1227

   micro avg       0.40      0.40      0.40      2997
   macro avg       0.32      0.52      0.31      2997
weighted avg       0.49      0.40      0.43      2997



In [201]:
scorer = make_scorer(cohen_kappa_score,weights='quadratic')
score = cross_val_score(clf,X_test,y_test,cv=10,scoring=scorer)

/home/gsm/miniconda3/envs/dsml/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/gsm/miniconda3/envs/dsml/lib/python3.6/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/home/gsm/miniconda3/envs/dsml/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/home/gsm/miniconda3/envs/dsml/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/gsm/miniconda3/envs/dsml/lib/python3.6/site-packages/sklearn

In [202]:
score.mean(),score.std()*2

(0.2971049084057112, 0.0643615307928938)

In [196]:
score

array([0.20367385, 0.26918343, 0.346144  , 0.19027638, 0.19015841,
       0.143218  , 0.24549992, 0.30486276, 0.38360098, 0.30101816])

In [130]:
#kfold = StratifiedKFold(n_splits=5, random_state=1337)

In [132]:
sub_df = pd.DataFrame()
sub_df['PetID'] = X_test['PetID']
sub_df['AdoptionSpeed'] = yh

In [133]:
sub_df.to_csv('submission.csv', index=False)

In [135]:
cat submission.csv

PetID,AdoptionSpeed
23cdf5f21,4
91efb738a,4
dbe30d86f,3
8b9d9a18b,3
c44a94d4b,2
0286dd665,2
70aa95825,2
95a89f5f2,4
222a3f408,1
0fe419dc3,4
97a020fe8,3
23f45d56f,4
911464bd8,2
8d6f10684,2
25c681109,3
05ab54e3c,2
368b2adc1,4
ab6c8aca6,2
7aa58b134,2
6d945f8ec,4
69ae5dba1,4
9180e09cb,2
8b396d0f4,1
171113c99,3
0bf2506a5,2
a2b2ff582,4
4f10d4a75,4
2c1c1b557,4
0a30968a3,2
5c82c5903,4
c135e081f,2
5b72e14bf,3
c1df421c4,4
db044e38b,4
bfb19deda,4
a37e46e35,2
6b6d2c65f,4
813658b4b,2
5fe2e44a2,4
6878a6fe0,2
c1ca516d7,2
e678bcc42,4
8e453a294,3
fe5ed81fd,2
ff706f8cc,4
d4beefae9,4
df3756483,4
230e1df8f,1
e95d902dc,4
043d8c534,4
283d93104,3
1347325db,2
eabb13cea,2
86d87d5f7,4
ea39ef3ae,4
c0503dd75,2
138c79a01,2
883e7d058,4
49310cec4,4
512bf1e8a,2
756d92515,3
4c4ba8c4e,2
944db63df,3
7feb1774c,1
46c5fd256,2
0ea0beb3c,4
4f279d30c,2
59c9f850a,4
2b6fe231b,4
0db02527a,2
d5c4ef4c0,2
4cdf29720,3
045543730,2
e204110aa,4
58bfbd845,4
a2656874e,4
bdbe7091f,1
5569b346f,4
d55faa0ff,2
35178a75a,4
edf74226c,4
fc52c0c8